# Face Verification Using Convolutional Neural Networks
- Task description: Design an end-to-end system for face verification with Convolutional Neural Networks (CNNs). Your system will be given two images as input and will output a score that quantifies the similarity between the faces in these images. This helps us decide whether the faces from the two images are of the same person or not.
- Evaluation: The Receiver Operating Characteristic (ROC) curve is created by plotting the True Positive Rate (TPR) against the False Positive Rate (FPR) at various threshold settings. The Area Under the Curve (AUC) for the ROC curve is equal to the probability that a classifier will rank a randomly chosen similar pair (images of same people) higher than a randomly chosen dissimilar one (images from two different people) (assuming 'similar' ranks higher than 'dissimilar' in terms of similarity scores).
- [Kaggle competition link](https://www.kaggle.com/c/11-785-fall-20-homework-2-part-2).

## Performance

- Epoch for the best result = 68.
- Ranking top 3% (5 out of 233) [[Kaggle leaderboard]](https://www.kaggle.com/c/11-785-fall-20-homework-2-part-2/leaderboard).
- Classification task:
    - training accuracy = 100%, loss = 0.0047.
    - validation accuracy = 89.74%, loss = 0.4977.
- Verification task:
    - validation AUC = 0.9712.
    - testing AUC = 0.9716 (at Kaggle).

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/gdrive/My\ Drive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle
# !cat ~/.kaggle/kaggle.json

In [ ]:
# !pip install --upgrade --force-reinstall --no-deps kaggle
# !kaggle datasets download -d cmu11785/20fall-hw2p2 -p /content/gdrive/My\ Drive/hw2p2/mydata

In [ ]:
# mypath = "/content/gdrive/My Drive/hw2p2/mydata/"
# import os
# os.chdir(mypath)  #change dir
# !ls

In [ ]:
# !unzip 20fall-hw2p2.zip -q

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import numpy as np
import torch
import torchvision   
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score
print(np.__version__)
print(torch.__version__)
cuda = torch.cuda.is_available()
cuda

1.18.5
1.6.0+cu101


True

## Load data (Torchvision DataSet and DataLoader)

In [ ]:
mypath = "/content/gdrive/My Drive/hw2p2/"
mydatapath = mypath + "mydata/"
class_data_path = mydatapath + "classification_data/"
verify_pairs_test_path = mydatapath + "verification_pairs_test.txt"
verify_pairs_val_path = mydatapath + "verification_pairs_val.txt"
result_path = mypath + "results/"

num_workers = 8

In [ ]:
## train data for classification / verification task
start_time = time.time()
train_set = torchvision.datasets.ImageFolder(root = class_data_path+"train_data/", 
                                             transform = torchvision.transforms.Compose([
                                                    torchvision.transforms.RandomHorizontalFlip(),
                                                    torchvision.transforms.ToTensor(),
                                                    ]))
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=8)
print("Time to load train data for classify task: ", (time.time() - start_time)/60, "mins")

In [ ]:
## validation data for classification task
start_time = time.time()
val_set = torchvision.datasets.ImageFolder(root = class_data_path+"val_data/", 
                                          transform = torchvision.transforms.Compose([
                                                    torchvision.transforms.ToTensor(),
                                                    ]))
val_loader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=num_workers)
print("Time to load val data for classify task: ", (time.time() - start_time)/60, "mins")

In [ ]:
# img0 = val_set.__getitem__(0)
# img0_x = img0[0].numpy()
# img0_y = img0[1]
# print(img0_x, img0_y)
# plt.imshow(img0_x.transpose((1,2,0)))

In [ ]:
def readFile(path, test):
    """Load verification data"""
    f = open(path, "rt").read().split('\n')
    img1s = []
    img2s = []
    labels = []
    for i, row in enumerate(f):
        row = row.split()
        if len(row) != 0:
            img1s.append(row[0])
            img2s.append(row[1])
            if not test:
                labels.append(int(row[2]))
            else:
                labels.append(-1)
    return img1s, img2s, labels

In [ ]:
class VerificationDataset(Dataset):
    """Dataset for Verification task"""
    def __init__(self, file_list1, file_list2, target_list=None):
        self.file_list1 = file_list1
        self.file_list2 = file_list2
        self.target_list = target_list

    def __len__(self):
        assert len(self.file_list1) == len(self.file_list2)
        return len(self.file_list1)

    def __getitem__(self, index):
        img1 = Image.open(mydatapath+self.file_list1[index])
        img1 = torchvision.transforms.ToTensor()(img1)
        img2 = Image.open(mydatapath+self.file_list2[index])
        img2 = torchvision.transforms.ToTensor()(img2)
        if self.target_list != None:
            label = self.target_list[index]
        else:
            label = -1
        return img1, img2, label

In [ ]:
## Read pair val data for verification task
verify_img1s_val, verify_img2s_val, verify_labels_val = readFile(verify_pairs_val_path, test=False)

## load val dataset and dataLoader for verification task
start_time = time.time()
verify_val_set = VerificationDataset(verify_img1s_val, verify_img2s_val, verify_labels_val)
verify_val_loader = DataLoader(verify_val_set, batch_size=128, shuffle=False, num_workers=num_workers, drop_last=False)
print("Time to load val data for verify task: ", (time.time() - start_time)/60, "mins")

## CNN Models

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self,in_channels,out_channels,stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # shortcut
        self.conv3_sc = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, bias=False)
        self.bn3_sc = nn.BatchNorm2d(out_channels)
   
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        residual = self.bn3_sc(self.conv3_sc(x))  # dotted line in Fig3 in ResNet paper
        out += residual
        out = F.relu(out)
        return out


class IdentityBlock(nn.Module):  
    """IdentityBlock has same in_channels and out_channels shape"""
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)

        self.conv2 = nn.Conv2d(channels, channels, kernel_size=1, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        residual = x   # solid line in Fig3 in ResNet paper
        out += residual
        out = F.relu(out)
        return out


class Resnet34(nn.Module):
    def __init__(self,classes=4000):
        super().__init__()
        # conv1
        self.conv1 = nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # conv2_x - *3
        self.conv2 = ConvBlock(64,64,stride=1)
        self.iden21 = IdentityBlock(64)
        self.iden22 = IdentityBlock(64)

        # conv3_x: down sample - stride 2, *4
        self.conv3 = ConvBlock(64,128,stride=2)
        self.iden31 = IdentityBlock(128)
        self.iden32 = IdentityBlock(128)
        self.iden33 = IdentityBlock(128)

        # conv4_x: down sample - stride 2, *6
        self.conv4 = ConvBlock(128,256,stride=2)
        self.iden41 = IdentityBlock(256)
        self.iden42 = IdentityBlock(256)
        self.iden43 = IdentityBlock(256)
        self.iden44 = IdentityBlock(256)
        self.iden45 = IdentityBlock(256)

        # conv5_x: down sample - stride 2, *3
        self.conv5 = ConvBlock(256,512,stride=2)
        self.iden51 = IdentityBlock(512)
        self.iden52 = IdentityBlock(512)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512,classes,bias = False)
        
    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))

        x = self.conv2(x)
        x = self.iden21(x)
        x = self.iden22(x)

        x = self.conv3(x)
        x = self.iden31(x)
        x = self.iden32(x)
        x = self.iden33(x)

        x = self.conv4(x)
        x = self.iden41(x)
        x = self.iden42(x)
        x = self.iden43(x)
        x = self.iden44(x)
        x = self.iden45(x)

        x = self.conv5(x)
        x = self.iden51(x)
        x = self.iden52(x)

        x = self.avgpool(x)

        x = torch.flatten(x, 1)
        out = self.fc(x)/torch.norm(self.fc.weight,dim=1)
        return out

## Training & validation

In [ ]:
def train_epoch(model, train_loader):
    model.to(device)
    
    model.train()
    start_time = time.time()
    avg_loss = 0.0
    for batch_num, (feats, labels) in enumerate(train_loader):
        feats, labels = feats.to(device), labels.to(device)

        optimizer.zero_grad()

        # NEW
        with torch.cuda.amp.autocast():
            outputs = model(feats)
            loss = criterion(outputs, labels.long())

        # NEW: Scales the loss, and calls backward() to create scaled gradients
        scaler.scale(loss).backward()
        # NEW: Unscales gradients and calls or skips optimizer.step()
        scaler.step(optimizer)
        # NEW: Updates the scale for next iteration
        scaler.update()

        # loss.backward()
        # optimizer.step()
        
        avg_loss += loss.item()

        if batch_num % 200 == 0:
            print('Batch: {}; avg loss: {:.4f}'.format(batch_num, avg_loss/50), 
                  "; time:{:.4f} mins".format((time.time()-start_time)/60))
            avg_loss = 0.0    
        
        torch.cuda.empty_cache()
        del feats
        del labels
        del loss

    print("*Epoch traing time:{:.4f} mins".format((time.time()-start_time)/60))
    train_loss, train_acc = test_classify(model, train_loader)
    return train_loss, train_acc


def test_classify(model, val_loader):
    start_time = time.time()
    with torch.no_grad():
        model.eval()
        test_loss = []
        accuracy = 0
        total = 0
        for batch_num, (feats, labels) in enumerate(val_loader):
            feats, labels = feats.to(device), labels.to(device)
            outputs = model(feats)

            _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
            pred_labels = pred_labels.view(-1)
            
            loss = criterion(outputs, labels.long())
            
            accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
            total += len(labels)
            test_loss.extend([loss.item()]*feats.size()[0])
            del feats
            del labels

    print("*Classify time:{:.4f} mins".format((time.time()-start_time)/60))
    return np.mean(test_loss), accuracy/total


def test_verify(model, val_loader, test=False):
    start_time = time.time()
    sim_preds = np.array([])
    sim_true = np.array([])
    with torch.no_grad():
        model.eval()
        for batch_num, (imgs1, imgs2, labels) in enumerate(val_loader):
            imgs1, imgs2 = imgs1.to(device), imgs2.to(device)
            
            if not test:
                sim_true = np.concatenate((sim_true, labels.numpy().reshape(-1)))
                del labels
            
            imgs1_out = model(imgs1)
            imgs2_out = model(imgs2)
            sim_pred = F.cosine_similarity(imgs1_out, imgs2_out) 
            sim_preds = np.concatenate((sim_preds, sim_pred.cpu().numpy().reshape(-1)))

            if batch_num % 50 == 0:
                print("Batch: {}; time:{:.4f} mins".format(batch_num, (time.time()-start_time)/60))
                if not test:
                    auc = roc_auc_score(sim_true, sim_preds)
                    print("***Verify task: val AUC = ", round(auc,4))
            del imgs1
            del imgs2

    # calculate auc at last
    if not test:
        auc = roc_auc_score(sim_true, sim_preds)
    else:
        auc = None
    print("*Verify time:{:.4f} mins".format((time.time()-start_time)/60))
    return sim_preds, sim_true, auc

## Begin training

In [ ]:
model = Resnet34() #Network(num_feats, hidden_sizes, num_classes) # 
criterion = nn.CrossEntropyLoss()

learningRate = 0.15 #1e-2
weightDecay = 5e-5
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=2, factor=0.85)

In [ ]:
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(init_weights)

In [ ]:
# last_epoch_trained_upon = 68

# model_version = "resNet34_aug_" + str(last_epoch_trained_upon)
# temp = torch.load(result_path + model_version)
# model.load_state_dict(temp['model_state_dict'])
# criterion.load_state_dict(temp['criterion_state_dict'])
# optimizer.load_state_dict(temp['optimizer_state_dict'])
# scheduler.load_state_dict(temp['scheduler_state_dict'])

In [ ]:
device = torch.device('cuda')  # 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

In [ ]:
# last_epoch_trained_upon = -1

In [ ]:
model_version = "resNet34_aug"
numEpochs = 100

# NEW: Creates once at the beginning of training
scaler = torch.cuda.amp.GradScaler()
# scaler.load_state_dict(temp['scaler_state_dict'])
for epoch in range(last_epoch_trained_upon+1, numEpochs):
    start_time0 = time.time()
    print(epoch)
    ## train
    train_loss, train_acc = train_epoch(model, train_loader) # about 1900 batchs
    print('***Classfy task: train loss: {:.4f}; train acc: {:.4f}'.format(train_loss, train_acc))
    ## eval mode
    # classification task
    val_loss, val_acc = test_classify(model, val_loader)
    scheduler.step(val_loss)
    print('***Classfy task: val loss: {:.4f}; val acc: {:.4f}'.format(val_loss, val_acc))
    # verification task
    _, _, verify_auc_val = test_verify(model, verify_val_loader)
    print("***Verify task: val AUC = ", round(verify_auc_val,4))
    print("*Whole epoch time:{:.4f} mins".format(epoch, (time.time()-start_time0)/60))   
    print('='*20)

    torch.save({
        "epoch": epoch,
        'model_state_dict': model.state_dict(),
        'criterion_state_dict' : criterion.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'train_loss': train_loss,
        "train_acc": train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'verify_auc_val': verify_auc_val
    }, result_path + model_version + "_" + str(epoch))

In [ ]:
!nvidia-smi

### Prediction of test data for verification task

In [ ]:
# best_epoch = 68  # auc 0.9714

# model_version = "resNet34_aug_" + str(best_epoch)
# temp = torch.load(result_path + model_version)
# model.load_state_dict(temp['model_state_dict'])

In [ ]:
# device = torch.device('cuda')  # 'cuda' if torch.cuda.is_available() else 'cpu'
# model.to(device)

In [ ]:
## Read pair test data for verification task
verify_img1s_test, verify_img2s_test, _ = readFile(verify_pairs_test_path, test=True)

## load test dataset and dataLoader for verification task
verify_test_set = VerificationDataset(verify_img1s_test, verify_img2s_test)
verify_test_loader = DataLoader(verify_test_set, batch_size=200, shuffle=False, num_workers=num_workers, drop_last=False)

In [ ]:
sim_preds_test, _, _ = test_verify(model, verify_test_loader, test=True)

In [ ]:
sim_preds_test[:10]

In [ ]:
# for i in range(len(sim_preds_test)):
#     if sim_preds_test[i] >= 0.8:
#         sim_preds_test[i] = 1
#     if sim_preds_test[i] <= 0.2:
#         sim_preds_test[i] = 0

In [ ]:
# sim_preds_test[:10]

In [ ]:
verify_test_file = open(verify_pairs_test_path, "rt").read().split('\n')

out_file = mypath + model_version + "res.csv"
with open(out_file, 'w') as w:
    w.write('id,Category\n')
    for i in range(len(sim_preds_test)):
        w.write(str(verify_test_file[i])+','+str(sim_preds_test[i])+'\n')

In [ ]:
print(len(sim_preds_test))

51835


In [ ]:
import pandas as pd
out_csv = pd.read_csv(out_file, sep='\t')
out_csv = np.array(out_csv)

In [ ]:
out_csv.shape

In [ ]:
out_csv

In [ ]:
!kaggle competitions submit -c 11-785-fall-20-homework-2-part-2 -f /content/gdrive/My\ Drive/hw2p2/resNet34_aug_68res.csv -m "Message"